In [1]:
# #!/bin/bash
# !curl -L -o ./credit-score-classification.zip\
#   https://www.kaggle.com/api/v1/datasets/download/parisrohan/credit-score-classification

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("train.csv")

/tmp/ipykernel_26564/2292805398.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")


In [4]:
column_idx_with_issue = 26
column_name = df.columns[column_idx_with_issue]
column_name

'Monthly_Balance'

In [5]:
df.dtypes

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                

In [6]:
df[column_name].map(type).value_counts()

Monthly_Balance
<class 'str'>      97132
<class 'float'>     2868
Name: count, dtype: int64

In [7]:
mask = df[column_name].map(type) == str
df.loc[mask, column_name].head(5)

0    312.49408867943663
1    284.62916249607184
2     331.2098628537912
3    223.45130972736786
4    341.48923103222177
Name: Monthly_Balance, dtype: object

Find rows that cannot be parsed as numeric (useful if column should be numeric):

In [8]:
# re-read that column as strings to see raw values
raw = pd.read_csv("train.csv", dtype=str, usecols=[column_name])
raw_vals = raw[column_name].dropna().head(200)
raw_vals.head(50)
# find values containing non-digit characters (excluding dot/minus)
import re
pattern = r'[^0-9\.\-]'
raw.loc[raw[column_name].str.contains(pattern, na=False), column_name].value_counts().head(50)

Monthly_Balance
__-333333333333333333333333333__    9
Name: count, dtype: int64

In [9]:
# value you found
bad_val = "__-333333333333333333333333333__"

# find how many and which row indices
mask = df[column_name] == bad_val
print("count:", mask.sum())
print("indices:", df.loc[mask].index.tolist())

# show the full rows for context
df.loc[mask].head(10)

count: 9
indices: [5545, 26177, 29158, 35570, 38622, 60009, 75251, 82918, 83255]


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
5545,0x367f,CUS_0x9885,February,Paul Dayl,17,186-49-9679,Developer,41086.4,3660.866667,3,...,Standard,113.98,34.826541,11 Years and 6 Months,Yes,145.544320,201.1429928919469,Low_spent_Medium_value_payments,__-333333333333333333333333333__,Standard
26177,0xaf63,CUS_0x5a90,February,Scuffhamq,45,264-53-1943,Engineer,29728.31,2551.359167,3,...,Good,917.0,38.198076,31 Years and 9 Months,No,34.496445,198.6115126951797,Low_spent_Large_value_payments,__-333333333333333333333333333__,Good
29158,0xc0d8,CUS_0x288d,July,Selamk,55,970-19-8342,Lawyer,30748.93,2572.410833,1,...,Good,860.97,40.634250,19 Years and 5 Months,No,43.727774,79.07180674947499,High_spent_Small_value_payments,__-333333333333333333333333333__,Standard
35570,0xe66c,CUS_0x85e9,March,Anna Driveri,45,288-62-7562,_______,20929.295_,1704.107917,5,...,_,858.09,33.943626,21 Years and 7 Months,No,13798.000000,220.30455197823662,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
38622,0xf84c,CUS_0x2b77,July,Longstretho,21,006-36-0111,Doctor,15167.62,1471.968333,4,...,Good,847.03,28.541848,NaN,NM,9.930809,128.87587207680795,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
60009,0x1759f,CUS_0xc06e,February,radenp,30,046-72-5387,Journalist,92114.68,7811.223333,3,...,Standard,820.52,39.089475,20 Years and 10 Months,Yes,54.219664,NaN,High_spent_Large_value_payments,__-333333333333333333333333333__,Standard
75251,0x1ceed,CUS_0x57f3,April,Kristen Haysq,29,577-85-0830,Mechanic,60904.59,NaN,3,...,Standard,2441.47,29.878716,12 Years and 11 Months,Yes,61.525140,394.8939566733383,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
82918,0x1fbd8,CUS_0x41bf,July,Doeringq,44,693-64-7611,Journalist,61990.52,4900.876667,10,...,Bad,3618.91,37.718591,13 Years and 8 Months,NM,355.143396,194.28348839148083,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
83255,0x1fdd1,CUS_0x2f7e,August,Sabina Zawadzkif,31,109-48-2095,Doctor,126353.04,10635.420000,4,...,_,859.77,28.442867,31 Years and 9 Months,No,167.043910,549.7661392945339,Low_spent_Medium_value_payments,__-333333333333333333333333333__,Poor


In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [11]:
import pandas as pd

# Example schema
dtypes = {
    "Customer_ID": "string",
    "Month": "string",
    "Name": "string",
    "Age": "float64",
    "SSN": "string",
    "Occupation": "string",
    "Annual_Income": "float64",
    "Monthly_Inhand_Salary": "float64",
    "Num_Bank_Accounts": "int",
    "Num_Credit_Card": "int",
    "Interest_Rate": "float64",
    "Num_of_Loan": "int",
    "Type_of_Loan": "string",
    "Delay_from_due_date": "int",
    "Num_of_Delayed_Payment": "int",
    "Changed_Credit_Limit": "float64",
    "Num_Credit_Inquiries": "float64",
    "Credit_Mix": "string",
    "Outstanding_Debt": "float64",
    "Credit_Utilization_Ratio": "float64",
    "Credit_History_Age": "string",
    "Payment_of_Min_Amount": "string",
    "Total_EMI_per_month": "float64",
    "Amount_invested_monthly": "float64",
    "Payment_Behaviour": "string",
    "Monthly_Balance": "float64",
    "Credit_Score": "string"
}

# Read raw CSV
df = pd.read_csv("train.csv")

# Apply schema with coercion
for col, typ in dtypes.items():
    if typ.startswith("float") or typ.startswith("int"):
        df[col] = pd.to_numeric(df[col], errors="coerce")  # invalid -> NaN
    elif typ == "string":
        df[col] = df[col].astype("string").replace(r"^\s*$", pd.NA, regex=True)


/tmp/ipykernel_26564/2872320392.py:35: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")


In [12]:
df[::40005].T

,0,40005,80010
ID,0x1602,0x10067,0x1ead0
Customer_ID,CUS_0xd40,CUS_0x41be,CUS_0x7d5e
Month,January,June,March
Name,Aaron Maashoh,Selyukha,<NA>
Age,23.0,35.0,19.0
SSN,821-00-0265,700-01-5042,063-39-1383
Occupation,Scientist,Musician,Journalist
Annual_Income,19114.12,NaN,NaN
Monthly_Inhand_Salary,1824.843333,6093.61,2409.553333
Num_Bank_Accounts,3,7,6


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  string 
 2   Month                     100000 non-null  string 
 3   Name                      90015 non-null   string 
 4   Age                       95061 non-null   float64
 5   SSN                       100000 non-null  string 
 6   Occupation                100000 non-null  string 
 7   Annual_Income             93020 non-null   float64
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               95215 non-null   float64
 13  Type_of_Loan              88592 non-null   st

In [14]:
df[::10000]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.120,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
10000,0x509a,CUS_0x229f,January,all Davidt,45.0,547-53-2472,_______,20261.850,1598.487500,7,...,Bad,1300.71,34.779107,14 Years and 2 Months,Yes,96.409635,124.514960,Low_spent_Small_value_payments,228.924155,Poor
20000,0x8b32,CUS_0x10c0,January,Scheyderx,38.0,658-53-1756,Architect,49454.130,4328.177500,8,...,Bad,1388.56,34.322164,12 Years and 9 Months,Yes,73.346359,304.051202,!@9#%8,345.420189,Standard
30000,0xc5ca,CUS_0x236b,January,Martinneo,25.0,575-63-7039,Manager,18393.510,1742.792500,9,...,_,2604.09,29.119227,4 Years and 3 Months,Yes,62.204931,233.032353,Low_spent_Small_value_payments,169.041966,Standard
40000,0x10062,CUS_0x41be,January,Selyukha,35.0,700-01-5042,Musician,72583.320,6093.610000,7,...,Standard,1019.44,37.220639,18 Years and 2 Months,Yes,104.016730,334.484535,Low_spent_Medium_value_payments,450.859735,Standard
50000,0x13afa,CUS_0x2646,January,ewich Billk,45.0,261-90-8736,Journalist,7807.285,606.607083,4,...,Standard,692.63,26.650258,<NA>,Yes,7.771075,31.082811,Low_spent_Small_value_payments,311.806822,Standard
60000,0x17592,CUS_0xa505,January,Jonathan Stempelz,41.0,959-35-8150,Engineer,34899.220,2764.268333,10,...,Standard,2103.76,30.962745,7 Years and 8 Months,Yes,78.967677,110.466661,Low_spent_Small_value_payments,376.992495,Poor
70000,0x1b02a,CUS_0x1d1b,January,Araxiac,20.0,#F%$D@*&8,Manager,152574.760,12547.563333,2,...,Good,NaN,38.632872,20 Years and 1 Months,No,205.484843,974.560691,Low_spent_Small_value_payments,364.710799,Standard
80000,0x1eac2,CUS_0x9df0,January,Timk,1642.0,846-31-4164,Entrepreneur,36115.920,2716.660000,0,...,_,886.95,27.797266,22 Years and 9 Months,No,78.052255,96.001442,High_spent_Small_value_payments,357.612303,Good
90000,0x2255a,CUS_0x3787,January,Kajimotoz,52.0,986-71-6476,Engineer,21270.790,1485.565833,6,...,Standard,782.15,39.263213,16 Years and 9 Months,Yes,48.481182,63.795843,High_spent_Small_value_payments,296.279558,Standard


### Convert column names to lower case

In [15]:
df.columns = df.columns.str.lower()

In [16]:
from IPython.display import display

In [17]:
for col in df.columns:
    if df[col].dtype == "string":
        display(f"Column Name: {col}, Unique count: {df[col].nunique()}")
        # display("Unique count: ", df[col].nunique())
        display(df[col].unique())
        display("-" * 20)

'Column Name: customer_id, Unique count: 12500'

<StringArray>
[ 'CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb',
 'CUS_0x95ee', 'CUS_0x284a', 'CUS_0x5407', 'CUS_0x4157', 'CUS_0xba08',
 ...
 'CUS_0x2654', 'CUS_0xb11c', 'CUS_0xad4f', 'CUS_0x51b3', 'CUS_0x2084',
 'CUS_0x372c',  'CUS_0xf16', 'CUS_0xaf61', 'CUS_0x8600', 'CUS_0x942c']
Length: 12500, dtype: string

'--------------------'

'Column Name: month, Unique count: 8'

<StringArray>
['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']
Length: 8, dtype: string

...

<StringArray>
[ 'High_spent_Small_value_payments',   'Low_spent_Large_value_payments',
  'Low_spent_Medium_value_payments',   'Low_spent_Small_value_payments',
 'High_spent_Medium_value_payments',                           '!@9#%8',
  'High_spent_Large_value_payments']
Length: 7, dtype: string

'--------------------'

'Column Name: credit_score, Unique count: 3'

<StringArray>
['Good', 'Standard', 'Poor']
Length: 3, dtype: string

'--------------------'

## Replace all incorrect values with `np.na`

### Data Quality Note: SSN Column
- **Total rows:** 100,000
- **Invalid SSN values removed:** 5,572 (5.5%)
- **Pattern enforced:** `\d{3}-\d{2}-\d{4}`
- **Action:** Replaced invalid entries with `pd.NA`
- **Justification:** SSN is an identifier, not a predictive feature. Invalid tokens (e.g., `#F%$D@*&8`) indicate data corruption.

In [18]:
df["ssn"] = df["ssn"].where(df["ssn"].str.match(r"^\d{3}-\d{2}-\d{4}$", na=False), pd.NA)

### Implement one-hot encoding for the column `type_of_loan`

#### Analyze a number of unique values in the `type_of_loan` column

In [26]:
df["type_of_loan"].hasnans

True

In [24]:
loan_types = set()

for val in df["type_of_loan"]:
    if pd.isna(val):
        continue

    normalized = val.replace(" and", ", ")
    types = normalized.split(",")
    for type in types:
        if type.strip() == "":
            continue
        loan_types.add(type.strip())

print(loan_types)
print(len(loan_types))

{'Student Loan', 'Credit-Builder Loan', 'Personal Loan', 'Mortgage Loan', 'Payday Loan', 'Debt Consolidation Loan', 'Home Equity Loan', 'Not Specified', 'Auto Loan'}
9


#### Apply one-hot encoding

In [27]:
for loan in sorted(loan_types):
    column_name = f"has_{loan.lower().replace(" ", "_").replace("-", "_")}"
    df[column_name] = df["type_of_loan"].apply(
        lambda s: 1 if pd.notna(s) and loan in s else 0
    ).astype("int8")

In [34]:
df = df.drop(["type_of_loan"], axis=1)

In [35]:
df.head()

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,...,credit_score,has_auto_loan,has_credit_builder_loan,has_debt_consolidation_loan,has_home_equity_loan,has_mortgage_loan,has_not_specified,has_payday_loan,has_personal_loan,has_student_loan
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,1,0,0,0,1,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,1,0,0,0,1,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.0,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,1,0,0,0,1,0
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,1,0,0,0,1,0
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,1,0,0,0,1,0
